In [2]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()


pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210722-1034-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [3]:
mt=hl.read_matrix_table(f'{lustre_dir}/variant_qc/MegaWESSanger_cohorts_AC_synonymous_filtered_kaitlin.mt')

In [4]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'id': str
    'proband': struct {
        s: str, 
        cohort: str
    }
    'father': struct {
        s: str, 
        cohort: str
    }
    'mother': struct {
        s: str, 
        cohort: str
    }
    'is_female': bool
    'fam_id': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: i

In [5]:
mt.info.AC.summarize()

Non-missing,2842 (100.00%)
Missing,0
Min Size,1
Max Size,5
Mean Size,1.17
Non-missing,3330 (100.00%)
Missing,0
Minimum,1
Maximum,140136
Mean,115.10
Std Dev,3194.81


In [6]:
mt.info.AC.show()

,,
locus,alleles,
locus<GRCh38>,array<str>,array<int32>
chr1:953842,"[""G"",""A""]",[3]
chr1:964132,"[""C"",""T""]",[8]
chr1:973580,"[""C"",""T""]",[2]
chr1:976244,"[""G"",""A""]",[4]
chr1:1090921,"[""G"",""A""]",[3]
chr1:1309580,"[""C"",""T"",""*""]","[2,1]"
chr1:1323205,"[""C"",""G"",""T""]","[2,1]"
chr1:1338408,"[""T"",""A"",""C""]","[7,1]"


In [7]:
mt.info.AC[0].summarize()

Non-missing,2842 (100.00%)
Missing,0
Minimum,1
Maximum,140136
Mean,133.28
Std Dev,3457.47


In [23]:
p=hl.plot.histogram(mt.info.AC[0])

In [24]:
show(p)

In [16]:
mt_trans=mt.filter_entries(mt.info.AC[0]==2)
mt_untrans=mt.filter_entries(mt.info.AC[0]==1)

Transmitted singletons

In [17]:
mt_trans_count=mt_trans.group_cols_by(mt_trans.id).aggregate(transmitted_singletons_count=hl.agg.count_where(
                               # (mt_trans.info.AC[0] == 2) &
                                (mt_trans.proband_entry.GT.is_non_ref()) &
                                (
                                (mt_trans.father_entry.GT.is_non_ref()  )
                                 |
                                (mt_trans.mother_entry.GT.is_non_ref())
                                )
                                ))
    

    
Total_transmitted_singletons=mt_trans_count.aggregate_entries(hl.agg.count_where(mt_trans_count.transmitted_singletons_count >0))
print(Total_transmitted_singletons)

685


Untransmitted singletons

In [18]:
mt_untrans_count = (mt_untrans.group_cols_by(mt_untrans.id).aggregate(
    untransmitted_singletons_count=hl.agg.count_where(
                   # (mt_untrans.info.AC[0] == 1) &
                    (mt_untrans.proband_entry.GT.is_hom_ref()) &
                    (
                    (mt_untrans.father_entry.GT.is_non_ref()) 
                    |
                    (mt_untrans.mother_entry.GT.is_non_ref())
                    )
                     )))
Total_untransmitted_singletons=mt_untrans_count.aggregate_entries(hl.agg.count_where(mt_untrans_count.untransmitted_singletons_count >0))
print(Total_untransmitted_singletons)

671


In [19]:
Ratio_transmitted_untransmitted=Total_transmitted_singletons/Total_untransmitted_singletons
print(Ratio_transmitted_untransmitted)

1.0208643815201193
